<a href="https://colab.research.google.com/github/ImpulsoGov/analises-publicas-saude/blob/main/2023_AnalisesImpacto/municipios_matriz_vizinhanca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install pygeos shapely fiona pyproj geopandas rtree psycopg2

In [ ]:
from getpass import getpass

import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.dialects.postgresql import CHAR
from sqlalchemy.engine.url import URL

In [ ]:
IBGE_GEOFTP = "https://geoftp.ibge.gov.br"
IBGE_MALHAS = IBGE_GEOFTP + "/organizacao_do_territorio/malhas_territoriais"
IBGE_MUNICIPIOS_2022 = (
    IBGE_MALHAS
    + "/malhas_municipais/municipio_2022/Brasil/BR/BR_Municipios_2022.zip"
)

In [ ]:
municipios_geo = gpd.read_file(IBGE_MUNICIPIOS_2022)
municipios_geo

In [ ]:
municipios_geo["id_sus"] = municipios_geo["CD_MUN"].apply(lambda id_ibge: id_ibge[:6])
municipios_geo

In [ ]:
relacoes_vizinhanca = []

for _, municipio in municipios_geo.iterrows():
    municipio_id_sus = municipio["id_sus"]
    vizinhos = municipios_geo[municipios_geo.touches(municipio["geometry"])]
    for _, vizinho in vizinhos.iterrows():
        relacoes_vizinhanca.append((municipio_id_sus, vizinho["id_sus"]))

relacoes_vizinhanca[:50]

In [ ]:
vizinhanca_df = pd.DataFrame(
    relacoes_vizinhanca,
    columns=["municipio_id_sus", "vizinho_municipio_id_sus"]
)

vizinhanca_df

In [ ]:
driver = "postgresql+psycopg2"
servidor="35.239.239.250"
porta="5432"
usuario=input("Insira o nome do usuário do banco de dados: ")
senha=getpass("Insira a senha para acesso ao banco de dados: ")
banco="principal"

bd_url = URL.create(
    drivername=driver,
    username=usuario,
    password=senha,
    host=servidor,
    port="5432",
    database=banco,
)

engine = create_engine(bd_url)
vizinhanca_df.to_sql(
    "ibge_municipios_matriz_vizinhanca",
    schema="dados_publicos",
    index=False,
    if_exists="replace",
    con=engine,
    dtype={
        "municipio_id_sus": CHAR(6),
        "vizinho_municipio_id_sus": CHAR(6),
    }
)


print("Ok!")